In [1]:
import email
import re

from collections import Counter
from dateutil.parser import parse
from pathlib import Path

In [2]:
def parse_emails(path):
    with open(path, 'r', encoding='windows-1252') as f:
        parsed_email = email.message_from_file(f)
    return parsed_email


def get_parsed_emails(paths):
    emails = []
    for i, path in enumerate(paths):
        eml = parse_emails(path)
        tms = int(parse(eml['Date']).timestamp())
        emails.append((i, eml, tms))
    return emails

#check to see if an email is part of chain by looking for the '-----Original Message-----' tag
#addtionally count the number of times the tag appears to get the depth of the chain
def is_chain(payload):
    chain_count = 0
    is_chain = False
    check_value = '-----Original Message-----'
    if check_value in payload:
        chain_count = payload.count(check_value)
        is_chain = True
    return (chain_count, is_chain)


#check to see if an email is forwarded by looking for 'fw' and 'fwd' tags
def is_forwarded(email):
    is_forwarded = False
    check_values = ['fw:', 'fwd:']
    for value in check_values:
        if value in email.get_payload().lower() or value in email['Subject'].lower():
            is_forwarded = True
    return is_forwarded


#get only the names from the x-headers in the e-mail
def parseXHeaders(header):
    if header is not None:
        return re.sub(r'(<.*?>,|<.*?>)', '|', header)[:-1]
    else:
        return header 


def get_emails_list(dir_path):
    clean_emails = []
    for path in dir_path.rglob('*.'):
        if 'all_documents' not in str(path.parent) and 'discussion_threads' not in str(path.parent):
            clean_emails.append(path)
    return clean_emails


def parse_emails(path):
    with open(path, 'r', encoding='windows-1252') as f:
        parsed_email = email.message_from_file(f)
    return parsed_email


def load_emails(dir_path):
    parsed_emails = []
    clean_emails = get_emails_list(dir_path)
    for i, path in enumerate(clean_emails):
        parsed_emails.append((i,parse_emails(path)))
    return parsed_emails


def preprocess_message(text):
    msg_end_pattern = re.compile('_{4,}.*|\n{3,}|<[^>]*>|-{4,}(.*)(\d{2}:\d{2}:\d{2})\s*(PM|AM)', re.MULTILINE)
    try:
        msg_end_iter = msg_end_pattern.search(text).start()
        # print('end of line:', msg_end_iter)
        message = text[:msg_end_iter]
    except AttributeError: # not a reply
        message = text
    return message


def remove_spaces(string):
    if string is not None:        
        string = re.sub('\s+', ' ', string)
        string = string.split(', ')
    return string


#
# Function: get_or_allocated_uid
# Arguments: name - string of a user email
# Returns: unique integer id
#
def get_or_allocate_uid(name):
     if name not in users:
         users[name] = len(users)
     return users[name]

#
# Function: get_or_allocate_tid
# Arguments: name - string of email subject line
# Returns: unique integer id
#
def get_or_allocate_tid(name):
    parsed_name = re.sub("(RE|Re|FWD|Fwd): ", "", name)
    if parsed_name not in threads:
        threads[parsed_name] = len(threads)
    return threads[parsed_name]

In [3]:
email_dir = Path(Path.cwd().parent, Path('data/raw/maildir'))

clean_emails = []
for path in email_dir.rglob('*.'):
    if 'all_documents' not in str(path.parent) and 'discussion_threads' not in str(path.parent):
        clean_emails.append(path)

emails = get_parsed_emails(clean_emails) 

## Test sender assumption

See if name of directory, v.g. `lay-k`, coincides with the sender of the emails from the `sent_items` or `_sent`, `sent`.

In [10]:
members = sorted([m.name for m in email_dir.glob('*')])

In [14]:
for path in email_dir.rglob('*.'):
    if 'lay-k' not in str(path.parent) and 'discussion_threads' not in str(path.parent):
        clean_emails.append(path)

True

In [24]:
str(path.parents[1]).split('/')[-1]

'rogers-b'